# Kernel Methods

###### COMP4670/8600 - Statistical Machine Learning - Tutorial

In this lab, we will predict the median value of houses in Boston using a quadratic basis and an equivalent kernel method.

### Assumed knowledge
- Linear regression (Week 2)
- Kernel methods (Week 4; Bishop 6.1, 6.2)


### After this lab, you should be comfortable with:
- Applying machine learning techniques with a non-linear basis
- Using kernel methods instead of a new basis
- Evaluating when using a kernel method would or would not be sensible

$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\dotprod}[2]{\langle #1, #2 \rangle}$

Setting up the environment

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

%matplotlib inline

## Loading the dataset
In this lab, we will use the same [dataset](https://machlearn.gitlab.io/sml2019/tutorials/02-dataset.csv) as in week 2, which describes the price of housing in Boston (see [description](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html)). 
We aim to predict the value of a home from other factors.
In this dataset, each row represents the data of one house. The first column is the value of the house, which is the target to predict. The remaining columns are features, which has been normalised to be in the range $[-1, 1]$. The corresponding labels of columns are

```'medv', 'crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat'```

Download the dataset. Read in the data using ```np.loadtxt``` with the optional argument ```delimiter=','```, as our data is comma separated rather than space separated. Remove the column containing the binary variable ```'chas'```.

Check that the data is as expected using ```print()```. It should have 506 rows (examples) and 13 columns (1 label and 12 features). Check that this is the case. 

Hint: use  assert.

In [ ]:
names = ['medv', 'crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat']

In [ ]:
# Solution
loaded_data = np.loadtxt('03-dataset.csv', delimiter=',')

# remove chas
column_idxes = list(range(len(names)))
chas_idx = names.index('chas')
wanted_columns = list(column_idxes)
wanted_columns.remove(chas_idx)
data = loaded_data[:,wanted_columns]
data_names = list(names)
data_names.remove('chas')

print(data_names)
print(np.array_str(data[:5], precision=3))
assert data.shape == (506,13)

## Before coding: Explain kernel methods, dual representations 

Please spend 5 minutes to explain to your neighbours what are kernel methods and dual representations.

Then find a piece of paper, derive how to from the linear regression model with regularised sum-of-squares error 
$$J(\mathbf{w}) = \frac{1}{2} \sum_{n=1}^N(\mathbf{w}^T \phi(\mathbf{x_n}) - t_n)^2 + \frac{\lambda}{2} \mathbf{w}^T\mathbf{w},$$
get to the dual representations 
$$J(\mathbf{a}) = \frac{1}{2} \mathbf{a}^T \mathbf{KKa} - \mathbf{a}^T \mathbf{Kt} + \frac{1}{2}\mathbf{t}^T \mathbf{t} + \frac{\lambda}{2} \mathbf{a}^T\mathbf{Ka},$$
where $\mathbf{K=\Phi \Phi}^T$ with elements $K_{nm} = \phi(\mathbf{x_n})^T\phi(\mathbf{x_m})$

## Overview

This notebook contains two parts. Part one is to implement linear regression with a basis function, for example, [a polynomial basis function of degree 2](https://en.wikipedia.org/wiki/Polynomial_kernel) as mentioned in week 2, we call it quadratic basis function below. The basis function can be understood as a feature mapping from raw input space into the new feature space. Part two is kernel regression, i.e. linear regression with kernel function $k(\mathbf{x,y}) = (\dotprod{\mathbf{x}}{\mathbf{y}} + 1)^2$. We'll see the kernel function implementation is equavilant of the quadratic basis function implementation.  

## Refresh: Linear regression

First remind yourself of what linear regression is by our implemention of linear regression with regularisation on the Boston dataset. Use 80% of the available data for training the model using maximum likelihood with regularisation (assuming Gaussian noise). The rest of the data is allocated to the test set.

Report the root mean squared error (RMSE) for the training set and the test set. We'll compare the results with the linear regression with basis function, and the linear regression with kernel function later on.

**TODO**: Implement the analytic solution of $\frac{\partial J(\mathbf{w})}{\partial\mathbf{w}}=0$ in the function  ```w_ml_regularised(Phi, t, l)```, where ```l``` stands for $\lambda$. Note that $$\Phi=
\begin{pmatrix}
    \phi(\mathbf{x_1})^T \\
    \phi(\mathbf{x_2})^T \\
    \vdots \\
    \phi(\mathbf{x_n})^T \\
\end{pmatrix},$$
where $\phi(\mathbf{x_i})$ denotes the feature vector for the $i$-th datapoint, and $n$ denotes the total number of datapoints.

In [ ]:
# Solution

def w_ml_regularised(Phi, t, l):
    """Produce the analytic solution of w given input features and labels."""

    return np.linalg.inv(l * np.eye(Phi.shape[1]) + Phi.T @ Phi) @ Phi.T @ t

def split_data(data, train_size):
    """Randomly split data into two groups. The first group is a fifth of the data."""
    np.random.seed(1)
    N = len(data)
    idx = np.arange(N)
    np.random.shuffle(idx)
    train_idx = idx[:int(train_size * N)]
    test_idx = idx[int(train_size * N):]

    # Assume label is in the first column
    X_train = data[train_idx, 1:]
    t_train = data[train_idx, 0]
    X_test = data[test_idx, 1:]
    t_test = data[test_idx, 0]
    
    return X_train, t_train, X_test, t_test

def rmse(X_train, t_train, X_test, t_test, w):
    """Return the RMSE for training and test sets"""
    N_train = len(X_train)
    N_test = len(X_test)

    # Training set error
    t_train_pred = np.dot(X_train, w)
    rmse_train = np.linalg.norm(t_train_pred - t_train) / np.sqrt(N_train)

    # Test set error
    t_test_pred = np.dot(X_test, w)
    rmse_test = np.linalg.norm(t_test_pred - t_test) / np.sqrt(N_test)

    return rmse_train, rmse_test

def lr(X, l, split_rate):
    """Return RMSE for the training set and the test set
    
    Parameters
    ------------------------------------------------------
    X: numpy matrix, whole dataset 
    l: float, regularisation parameter
    split_rate: int, the percent of training dataset
    
    Returns
    ------------------------------------------------------
    train_rmse: float, RMSE for training set
    test_rmse: float, RMSE for testing set
    """
    X0 = np.ones((data.shape[0], 1))
    X = np.hstack((X, X0))

    X_train, t_train, X_test, t_test = split_data(X, split_rate)
    w_reg = w_ml_regularised(X_train, t_train,l)

    train_rmse, test_rmse = rmse(X_train, t_train, X_test, t_test, w_reg)
    return train_rmse, test_rmse

train_rmse, test_rmse = lr(data, 1.1, 0.8)
print("Regression: RMSE with regularisation: Train {:.6f}, Test {:.6f}".format(train_rmse, test_rmse))
print("Expected value: Train 4.74, Test 4.89")

## Part 1: Linear regression with quadratic basis function

Let $X \in \RR^{n \times d}$ be the data matrix containing n datapoints $\mathbf{x_i} \in \RR^{d}$. We can choose to train and test using the raw data $X$ as the input to our model as the above method. Alternatively, we could use $$\Phi= [\mathbf{\phi(x_1)}, \mathbf{\phi(x_2)}, ..., \mathbf{\phi(x_n)}]^T \in \RR^{n \times m},$$ where $\mathbf{\phi(x_i)}$ is some transformation of $\mathbf{x_i}$, m is the dimension of $\mathbf{\phi(x_i)}$.

For this lab, write $\mathbf{x_i} = (x_{i,1},\dots,x_{i,d})$. Let
$$
\phi(\mathbf{x_i}) = (x_{i,1}^2, x_{i,2}^2, \ldots, x_{i,d}^2, \sqrt{2}x_{i,1} x_{i,2}, \ldots, \sqrt{2}x_{i,d-1}x_{i,d}, \sqrt{2}x_{i,1}, \ldots, \sqrt{2}x_{i,d}, 1).
$$
Note that $\phi(\mathbf{x_i})$ is all quadratic functions of elements of $\mathbf{x_i}$ and 1 (The $\sqrt{2}$ coefficients are for normalisation for later in the lab).
We say that we are using a *quadratic basis function*.

Train and test the data with ```quadratic_lr```, report the RMSE for the training set and the test set.

**TODO**:  
1. write a function called ```phi_quadratic``` with a single datapoint $\mathbf{x_i}$ as input and the quadratic basis function $\phi(\mathbf{x_i})$ as output.  
2. write a function called ```feature_map``` with raw data matrix $X$ as input and $\Phi$ as output by using ```phi_quadratic``` for each datapoint. 
3. in function ```quadratic_lr```, make use of previous functions and give the analytic solution for $\mathbf{w}$.

In [ ]:
# Solution

def phi_quadratic(x):
    """Compute phi(x) for a single training example using quadratic basis function."""
    D = len(x)
    # Features are (1, {x_i}, {cross terms and squared terms})
    # Cross terms x_i x_j and squared terms x_i^2 can be taken from upper triangle of outer product of x with itself
    utri = np.sqrt(2)*np.hstack((x, np.outer(x, x)[np.triu_indices(D, k=1)]))
    diag = x * x
    res = np.hstack((1,diag, utri))
    
    return res

def feature_map(X):
    """Return the matrix of the feature map."""
    num_ex, num_feat = X.shape
    D = int(1+num_feat+num_feat*(num_feat+1)/2)
    Phi = np.zeros((num_ex, D))
    np.shape(Phi)
    for ix in range(num_ex):
        Phi[ix,:] = phi_quadratic(X[ix,:])
    return Phi

def quadratic_lr(X, l, split_rate):
    """Return RMSE for the training set and the test set
    
    Parameters
    ------------------------------------------------------
    X: numpy matrix, whole dataset 
    l: float, regularisation parameter
    split_rate: int, the percent of training dataset
    
    Returns
    ------------------------------------------------------
    train_rmse: float, RMSE for training set
    test_rmse: float, RMSE for testing set
    """
    
    X_train, t_train, X_test, t_test = split_data(X, split_rate)
    # alternatively: use train_test_split
    # X_train, X_test, t_train, t_test = train_test_split(X[:,1:], X[:, 0], test_size = 1-split_rate, random_state = 42)
    w_reg = w_ml_regularised(feature_map(X_train), t_train,l)

    train_rmse, test_rmse = rmse(feature_map(X_train), t_train, feature_map(X_test), t_test, w_reg)
    return train_rmse, test_rmse

train_rmse, test_rmse = quadratic_lr(data, 1.1, 0.8)
print("Quadratic basis: RMSE with regularisation: Train {:.6f}, Test {:.6f}".format(train_rmse, test_rmse))
print("Expected value: Train 2.79, Test 3.34")

## Part 2: Linear regression with a kernel

### Computing a basis transformation as an inner product

Define $k(\mathbf{x,y}) = (\dotprod{\mathbf{x}}{\mathbf{y}} + 1)^2$, where $\mathbf{x,y} \in \mathbb{R}^2$. One way to verify $k(\mathbf{x,y})$ is a kernel function is to write this as an inner product of a verctor valued function evaluated at $\mathbf{x}$ and $\mathbf{y}$. That is, show we have $k(\mathbf{x}, \mathbf{y}) = \dotprod{\phi(\mathbf{x})}{\phi(\mathbf{y})}$ and specify what is $\phi(\mathbf{x}), \phi(\mathbf{y})$.

### Solution

\begin{align}
k(\mathbf{x,y}) &= (\dotprod{\mathbf{x}}{\mathbf{y}} + 1)^2 = (\mathbf{x}^T\mathbf{y}+ 1)^2 = (x_1y_1 + x_2y_2 + 1)^2\\
&= x_1^2y_1^2 + x_2^2y_2^2 + 2x_1x_2y_1y_2 + 2x_1y_1 + 2x_2y_2 + 1\\
&= (x_1^2, x_2^2, \sqrt{2} x_1x_2, \sqrt{2}x_1, \sqrt{2}x_2, 1)(y_1^2, y_2^2, \sqrt{2} y_1y_2, \sqrt{2}y_1, \sqrt{2}y_2, 1)^T
\end{align}
Let $\phi(\mathbf{x}) = (x_1^2, x_2^2, \sqrt{2} x_1x_2, \sqrt{2}x_1, \sqrt{2}x_2, 1)^T$, $\phi(\mathbf{y}) = (y_1^2, y_2^2, \sqrt{2} y_1y_2, \sqrt{2}y_1, \sqrt{2}y_2, 1)^T$, then we have $k(\mathbf{x,y}) = \phi(\mathbf{x})^T\phi(\mathbf{y}) = \dotprod{\phi(\mathbf{x})}{\phi(\mathbf{y})}$

Then convince yourself that, for $X \in \RR^{n \times d}, Y \in \RR^{m \times d}$, then $K = (X Y^T  + 1)^2 \in \RR^{n \times m}$ (addition and square term are element-wise) contains elements as kernel function between each pair of datapoints of X and Y,
$$K_{ij} = \phi(\mathbf{x_i})^T \phi(\mathbf{y_j}) = k(\mathbf{x_i}, \mathbf{y_j}).$$

### Kernelised Regression
**TODO**:
Write the function ```kernel_quadratic``` which takes $X$, $Y$  as input and $K$ as output.

In [ ]:
# Solution

def kernel_quadratic(X, Y):
    dot_product = np.dot(X, Y.T)
    return (dot_product+1)**2

**TODO**:
Complete the function ```kernelised_lr``` with raw data matrix $X$ as input and root mean squared error (RMSE) for the training set and the test set as output. Inside of the function, make use of ```kernel_quadratic``` to apply dual representation, and calculate the predicted labels for training data and testing data repectively.

In [ ]:
# Solution

def kernelised_lr(X, l, split_rate):
    """Return RMSE for the training set and the test set
    
    Parameters
    ------------------------------------------------------
    X: numpy matrix, whole dataset 
    l: float, regularisation parameter
    split_rate: int, the percent of training dataset
    
    Returns
    ------------------------------------------------------
    rmse_train: float, RMSE for training set
    rmse_test: float, RMSE for testing set
    """
    
    X_train, t_train, X_test, t_test = split_data(X, split_rate)
    N_train = X_train.shape[0]
    N_test = X_test.shape[0]
    
    K_train = kernel_quadratic(X_train, X_train)
    K_test = kernel_quadratic(X_test, X_train)
    
    # critical points
    a =  np.linalg.inv(K_train + l * np.eye(N_train)).dot(t_train)
    
    # Training set error
    t_train_pred = np.dot(K_train, a)
    rmse_train = np.linalg.norm(t_train_pred - t_train) / np.sqrt(N_train)

    # Test set error
    t_test_pred = np.dot(K_test, a)
    rmse_test = np.linalg.norm(t_test_pred - t_test) / np.sqrt(N_test)

    return rmse_train, rmse_test

train_rmse, test_rmse = kernelised_lr(data, 1.1, 0.8)
print("Kernelised Regression: RMSE with regularisation: Train {:.6f}, Test {:.6f}".format(train_rmse, test_rmse))
print("Expected value: Train 2.79, Test 3.33")

## Time complexity

Compare the above two methods (quadratic basis function, kernel method) in terms of time complexity.

In terms of time complexity, is using a kernel method suitable in this case? What are potential advantages of using a kernel method? Disadvantages? 

### Solution

Assume the dataset with feature mapping is $N \times M$. 

The linear regression with quadratic basis function requires to invert an $M \times M$ matrix, while the linear regression with kernel method requires to invert a $N \times N$ matrix. In our case, the feature mapping space is 91 (which is 12 + (12 * 11)/2 + 12 + 1), which is much smaller than the number of data points (506 in total). Thus a kernel method might not suit our case in terms of time complexity, but when N is smaller than M, kernel method is more efficient.

## Textbook Questions (Optional)
These questions are hand picked to both be of reasonable difficulty and demonstrate what you are expected to be able to solve. The questions are labelled in Bishop as either $\star$, $\star\star$, or $\star\star\star$ to rate its difficulty.

- **Question 6.1**: Understand dual formulation (Difficulty $\star\star$, simple algebraic derivation)
- **Question 6.2**: Dual formulation for perceptron learning algorithm (Difficulty $\star\star$, simple algebraic derivation)
- **Question 6.11**: You may want to use Taylor expansion to represent term $\exp(\mathbf{x}^T\mathbf{x}'/2\sigma^2)$ (Difficulty $\star$)
- **Question 6.12**: To prove $k\left(A_{1}, A_{2}\right)=2^{\left|A_{1} \cap A_{2}\right|}=\phi\left(A_{1}\right)^{T} \phi\left(A_{2}\right)$. (Difficulty $\star\star$)
- **Question 6.13**: Use chain rule to represent $g(\varphi(\theta), x)$ (Difficulty $\star$)
- **Question 7.6**: (Difficulty $\star$, simple algebraic derivation)
- **Question 7.7**: (Difficulty $\star$, simple algebraic derivation)